In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk
from sklearn import svm
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
import sklearn.ensemble as ske
import seaborn as sns
%matplotlib inline

In [278]:
df = pd.read_csv('./data/application_train.csv', sep=',', header=0)
print(df.info())
df.head(1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Columns: 122 entries, SK_ID_CURR to AMT_REQ_CREDIT_BUREAU_YEAR
dtypes: float64(65), int64(41), object(16)
memory usage: 286.2+ MB
None


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0


In [279]:
print("Number of columns with missing data: " + str(len(df.columns[df.isnull().any()].tolist())))

Number of columns with missing data: 67


In [2]:
def replace_nulls(df):
    
    columns = list(df.columns.values)
    
    for col in columns:
        # Replace numeric values with mean
        if (np.issubdtype(df[col], np.number)):
            df[col].fillna(df[col].mean(), inplace = True)
        
        # Replace categorical and boolean values with highest occuring value
        else:
            df[col].fillna(df[col].value_counts().index[0], inplace = True)
    return df

In [281]:
df = replace_nulls(df)
print("Number of columns with missing data: " + str(len(df.columns[df.isnull().any()].tolist())))

Number of columns with missing data: 0


In [3]:
def process_categorical(df):
    
    categorical_columns = ['NAME_CONTRACT_TYPE',
                           'CODE_GENDER',
                           'FLAG_OWN_CAR',
                           'FLAG_OWN_REALTY',
                           'NAME_TYPE_SUITE',
                           'NAME_INCOME_TYPE',
                           'NAME_EDUCATION_TYPE',
                           'NAME_FAMILY_STATUS',
                           'NAME_HOUSING_TYPE',
                           'OCCUPATION_TYPE',
                           'WEEKDAY_APPR_PROCESS_START',
                           'ORGANIZATION_TYPE',
                           'FONDKAPREMONT_MODE',
                           'HOUSETYPE_MODE',
                           'WALLSMATERIAL_MODE',
                           'EMERGENCYSTATE_MODE']
    
    # Significance of categories for target, will later try more with this
    #for col in categorical_columns:
    #   df.groupby(col).mean()['TARGET'].sort_values(ascending=False)


    df[categorical_columns] = df[categorical_columns].apply(preprocessing.LabelEncoder().fit_transform)
    
    return df

In [283]:
df = process_categorical(df)
df.head(1)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,0,1,0,1,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0


In [4]:
def preprocess_data(df):
    df = replace_nulls(df)
    df = process_categorical(df)
    df = df.drop('SK_ID_CURR', axis=1)
    
    return df

In [5]:
df = pd.read_csv('./data/application_train.csv', sep=',', header=0)
df = preprocess_data(df)
print("Number of columns with missing data: " + str(len(df.columns[df.isnull().any()].tolist())))
df.head(1)

Number of columns with missing data: 0


,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,1,0,1,0,1,0,202500.0,406597.5,24700.5,351000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0


In [6]:
def split_data(df):
    df_train = df.sample(frac = 0.9, random_state = 42)
    df_val = df.drop(df_train.index)  
    
    X_train = df_train.drop(['TARGET'], axis=1).values
    y_train = df_train['TARGET'].values
    
    X_val = df_val.drop(['TARGET'], axis=1).values
    y_val = df_val['TARGET'].values
    
    return X_train, y_train, X_val, y_val

In [7]:
X_train, y_train, X_val, y_val = split_data(df)

In [8]:
def train_clf(clf, X_train, y_train, X_val, y_val):
    
    clf.fit (X_train, y_train)
    
    y_act, y_pred = y_val, clf.predict(X_val)
    scores = cross_val_score(clf, X_val, y_val, cv=10, scoring='accuracy')
    print("Validation Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))
    
    return clf

In [9]:
rf_clf = ske.RandomForestClassifier(n_estimators=50)
rf_clf = train_clf(rf_clf, X_train, y_train, X_val, y_val)

Validation Accuracy: 0.92 (+/- 0.00)


In [21]:
def predict(df, clf, export_path):
    
    # Extract Ids
    ids = df['SK_ID_CURR'].values
    
    # Make predictions
    df = preprocess_data(df)
    X = df.values
    y = clf.predict(X)
    
    # Combine ids and predictions
    y = np.column_stack((ids, y))
    
    # Restore pandas df
    df_pred = pd.DataFrame(y)
    df_pred.columns = ["SK_ID_CURR", "TARGET"]
    
    # Export
    df_pred.to_csv(export_path, sep=',', index=False)
    
    return df_pred

In [ ]:
df_test = pd.read_csv('./data/application_test.csv', sep=',', header=0)
df_pred = predict(df_test, rf_clf, './predictions/rf_pred.csv')

In [ ]:
print(df_pred.sum())